# Schwarzschild orbits
Star mass $M$, particle mass $m$, and total energy $E\leq m$

\begin{align}
ds = \big(E^2-m^2+\frac{2Mm^2}{r}\big)\Big(\frac{dr^2}{\big(1-\frac{2M}{r}\big)^2}+ \frac{r^2d\Phi^2}{1-\frac{2M}{r}}\Big)
\end{align}
\begin{align}
&dr=\frac{1}{r}(dx+dy),\qquad d\Phi = -\frac{y}{r^2}dx+\frac{x}{r^2}dy\\
&dr^2 = \frac{1}{r^2}(dx^2+2dxdy+dy^2),\qquad d\Phi^2 = \frac{1}{r^4}(y^2dx^2-2xydxdy+x^2dy^2)
\end{align}
Thus, with $g:=E^2-m^2+\frac{2Mm^2}{r}$ and $f:= \big(1-\frac{2M}{r}\big)^{-1}$
\begin{align}
ds = \frac{fg}{r^2}\Big( (f\,x^2+y^2)\,dx^2 + 2(f-1)xy\,dxdy+(x^2+f\,y^2)\,dy^2 \Big)
\end{align}

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from sympy import symbols, diff, Matrix, simplify
import sympy as sp

H = -1.5
L = 0.5
M = 0.0025
m = 1/M**0.5
E = (2*H+m**2)**0.5

X,Y = symbols('x y')
xvec = [X,Y]
r = sp.sqrt(X**2+Y**2)
g = E**2 - m**2 + 2 * M * m**2 / r
f = 1 / (1 - 2 * M / r)
G = simplify(f * g / r**2 * Matrix(2,2, [f * X**2 + Y**2, (f - 1) * X * Y,
                        (f - 1) * X * Y, X**2 + f * Y**2]))


Ginv = simplify(1/G.det()*G.adjugate())
chr1 = [simplify(0.5*(diff(G[k,j],xvec[i])+diff(G[i,k],xvec[j])-diff(G[i,j],xvec[k]))) for i in range(2) for j in range(2) for k in range(2)]
chr2 = [(sum(Ginv[k,p]*chr1[4*i+2*j+p] for p in range(2))) for i in range(2) for j in range(2) for k in range(2)]

# define right-hand side (s... time, v=[gamma^1,gamma^2,v^1,v^2])
def rhs(s, v): 
    chr111 = chr2[4*0+2*0+0].subs(Y, v[1]).subs(X,v[0])
    chr121 = chr2[4*0+2*1+0].subs(Y, v[1]).subs(X,v[0])
    chr221 = chr2[4*1+2*1+0].subs(Y, v[1]).subs(X,v[0])
    chr112 = chr2[4*0+2*0+1].subs(Y, v[1]).subs(X,v[0])
    chr122 = chr2[4*0+2*1+1].subs(Y, v[1]).subs(X,v[0])
    chr222 = chr2[4*1+2*1+1].subs(Y, v[1]).subs(X,v[0])
    return [v[2],v[3],-v[2]**2*chr111-2*v[2]*v[3]*chr121-v[3]**2*chr221,-v[2]**2*chr112-2*v[2]*v[3]*chr122-v[3]**2*chr222]


Tend = 20
refval = solve_ivp(rhs, (0, Tend), [0.5,0,-0.2,0.5],t_eval=np.linspace(0, Tend,400))#, method="DOP853")
plt.plot(refval.y.T[:,0],refval.y.T[:,1])
plt.gca().add_patch(plt.Circle((0, 0), 0.05, color='r'))
plt.axis('equal')
plt.show()

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt
from time import time


H = -1.5
L = 0.5
M = 0.0025
m = 1/M**0.5
E = (2*H+m**2)**0.5
r = sqrt(x**2+y**2)


g = E**2 - m**2 + 2 * M * m**2 / r
f = 1 / (1 - 2 * M / r)
Gex = f * g / r**2 * CF((f * x**2 + y**2, (f - 1) * x * y,
                        (f - 1) * x * y, x**2 + f * y**2),dims=(2,2))

shape = Circle((0,0),0.8).Face()
mesh = Mesh(OCCGeometry(shape,dim=2).GenerateMesh(maxh=0.1))
Draw(Norm(Gex), mesh)